In [ ]:
# %% [markdown]
# # Lab 9: The ReAct Agent Pattern
#
# **Goal:** Build a simple agent that follows the **ReAct (Reasoning and Acting)** pattern. This is a fundamental pattern for creating agents that can use tools to answer questions or accomplish tasks.
#
# ---

# %% [markdown]
# ## Part A: Understanding ReAct
#
# ReAct is a simple but powerful loop that an agent follows to achieve a goal.
#
# 1.  **Reason:** The agent thinks about its goal and the information it has. It forms a plan or decides what to do next. This "thought" is a crucial step.
# 2.  **Act:** Based on its reasoning, the agent decides to take an action. This action is often the use of a "tool" (e.g., a search engine, a calculator, a database query).
# 3.  **Observe:** The agent receives the result from the tool (the "observation").
# 4.  **Repeat:** The agent takes this new observation, goes back to the **Reason** step, and continues the loop until it has enough information to achieve its goal.
#
# ### Example Flow for Our Recruiter Agent:
#
# * **Goal:** Assess the skills of a candidate.
# * **Reasoning (Thought):** "The resume mentions a GitHub profile. I should check it to verify their technical skills."
# * **Action (Tool Call):** `search_github(username="john_dev")`
# * **Observation (Tool Result):** "Found a GitHub profile with 50 Python repositories. Very active."
# * **Reasoning (Thought):** "The GitHub profile confirms strong Python skills. I can now finalize my assessment."
# * **Action:** End the loop and provide the final answer.
#
# ---

# %% [markdown]
# ## Part B: Simple ReAct Implementation
#
# We will build a graph that implements this loop. Our agent will have a simulated "tool" to check a candidate's experience.

# %%
# Setup
!pip install langgraph --quiet
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal

# 1. Define the State
class ReActState(TypedDict):
    candidate_info: str
    thought: str
    action: str
    observation: str
    step_count: int
    final_assessment: str

# 2. Define the "Tool"
def check_experience_tool(candidate_info: str) -> str:
    """A simulated tool to verify a candidate's experience."""
    print(f"---TOOL: Checking experience for '{candidate_info}'---")
    if "5 years" in candidate_info.lower() and "senior" in candidate_info.lower():
        return "Experience claims verified. Projects listed are consistent with a senior role."
    else:
        return "Experience claims could not be verified. More information is needed."

# 3. Define the Graph Nodes
def reasoning_step(state: ReActState) -> dict:
    """The agent thinks about what to do."""
    print(f"\n---NODE: REASONING (Step {state['step_count']})---")

    # On the first step, the agent decides to use the tool.
    if state['step_count'] == 0:
        thought = "I need to verify the candidate's experience claims. I will use the experience checking tool."
        action = "check_experience"
    # On subsequent steps, it uses the tool's observation to make a final decision.
    else:
        thought = "I have the verification result. I can now make a final assessment."
        action = "finalize"

    print(f"Thought: {thought}")
    return {"thought": thought, "action": action}


def acting_step(state: ReActState) -> dict:
    """The agent takes an action (uses a tool)."""
    print(f"---NODE: ACTING---")

    if state['action'] == "check_experience":
        observation = check_experience_tool(state['candidate_info'])
    else:
        observation = "No tool used."

    print(f"Observation: {observation}")
    return {"observation": observation, "step_count": state['step_count'] + 1}

def finalize_assessment(state: ReActState) -> dict:
    """The agent provides its final assessment."""
    print("---NODE: FINALIZING---")
    if "verified" in state['observation'].lower():
        assessment = "Strong candidate. Experience is verified. Recommend for interview."
    else:
        assessment = "Potential issues with experience claims. Recommend for manual HR review."
    print(f"Final Assessment: {assessment}")
    return {"final_assessment": assessment}

# 4. Define the Conditional Edge
def should_continue(state: ReActState) -> Literal["continue", "end"]:
    """Decides whether to continue the loop or end."""
    # If the action is to finalize, we end the loop.
    if state['action'] == 'finalize':
        return "end"
    # Otherwise, we continue to the acting step.
    else:
        return "continue"

# %% [markdown]
# ### Assembling the ReAct Graph
#
# The key to the ReAct pattern is a conditional edge that creates a loop, sending the agent back to the `reason` node after it `acts` and `observes`.

# %%
# Build the ReAct graph
workflow = StateGraph(ReActState)

# Add nodes
workflow.add_node("reason", reasoning_step)
workflow.add_node("act", acting_step)
workflow.add_node("finalize", finalize_assessment)

# Add edges
workflow.set_entry_point("reason")

# This conditional edge is the core of the ReAct loop.
# From the 'reason' node, we check if we should continue or end.
workflow.add_conditional_edges(
    "reason",
    should_continue,
    {
        "continue": "act", # If continue, go to the tool-using 'act' node
        "end": "finalize"   # If end, go to the 'finalize' node
    }
)
# After acting, we always go back to reasoning with the new information.
workflow.add_edge("act", "reason")
workflow.add_edge("finalize", END)

# Compile the agent
react_agent = workflow.compile()
print("ReAct agent compiled successfully.")

# %% [markdown]
# ---
# ## Part C: Test the ReAct Agent
#
# Let's run our agent with a candidate to see the ReAct loop in action.

# %%
# Test Case: A verifiable senior candidate
test_state = ReActState(
    candidate_info="Jane Doe is a senior developer with 5 years of experience.",
    step_count=0
)

# Invoke the agent
final_state = react_agent.invoke(test_state)

print("\n---FINAL AGENT STATE---")
print(final_state)